In [1]:
from datasets import load_dataset

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

from model import Model

torch.set_printoptions(linewidth=1000000)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


# Load trained model
Load the model from a trained checkpoint

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [3]:
model = Model.load_from_checkpoint('flan-t5-base-batch4.ckpt')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base', truncation_side='left')
model.eval()

Model(
  (model): PeftModelForSeq2SeqLM(
    (base_model): LoraModel(
      (model): T5ForConditionalGeneration(
        (shared): Embedding(32128, 768)
        (encoder): T5Stack(
          (embed_tokens): Embedding(32128, 768)
          (block): ModuleList(
            (0): T5Block(
              (layer): ModuleList(
                (0): T5LayerSelfAttention(
                  (SelfAttention): T5Attention(
                    (q): Linear(
                      in_features=768, out_features=768, bias=False
                      (lora_dropout): ModuleDict(
                        (default): Dropout(p=0.05, inplace=False)
                      )
                      (lora_A): ModuleDict(
                        (default): Linear(in_features=768, out_features=16, bias=False)
                      )
                      (lora_B): ModuleDict(
                        (default): Linear(in_features=16, out_features=768, bias=False)
                      )
                      (lora_embeddi

# Configure dialog with model
Creating an action to easily "communicate" with the model

In [4]:
context = {}

def dialog(topic, prompt):
    with torch.no_grad():
        current_context = ''

        if topic in context:
            context[topic].append(prompt)
        else:
            context[topic] = []
            context[topic].append(prompt)

        current_context = ' '.join(context[topic])
        tokenized_input = tokenizer(current_context, return_tensors="pt").to(device)

        generated = model.model.generate(**tokenized_input, do_sample=True, top_p=0.9, max_new_tokens=200, no_repeat_ngram_size =5)
        print(generated.dtype)
        generated_text = tokenizer.decode(generated[0], skip_special_token=True)
        context[topic].append(generated_text)

        return generated_text
    

In [5]:
print(dialog('eval', 'What is the white house?'))

torch.int64
<pad> The White House is a federal government that serves the American public. It is the centerpiece of the Obama administration's budget and strategy, and is backed by the White House's extensive range of policies and strategies, including tax cuts, tax reform, and reform of the economy. The White House provides a unique and expansive range of policies, policy ideas, and programs to help Americans achieve their goals, including tax cuts and reforms. It is also a venue for events and events that make Americans feel connected to the White House and its people. The White House offers a diverse range of policies, strategies, and strategies to help Americans achieve the goals they set out to achieve, such as reducing tax burdens, reducing bureaucracy, and providing an environment conducive to community and community engagement. It is also home to several groups, including the American Civil Liberties Union, National Human Rights League, the American Civil Libertarian Union, and

In [6]:
print(dialog('eval', 'Who lives there?'))

torch.int64
<pad> The White House is located on the third floor of the White House, with its entrance on a second floor facing the Capitol and its adjacent offices. The White House is also home to the Office of General Counsel, the Office of Management and Budget, the Office of International Trade, the White House Research Bureau, and the Office of International Relations. The White House also is home to the Office and Research Center, both of which are focused on the White House's mission and mission. The White House has a large and diverse collection of books, documents, and materials, including a comprehensive collection of books, journals, and publications on issues of international interest. The White House maintains a diverse collection of documents, such as the Civil War books, the Civil War textbook, and the White House Collection of World Records. The White House's Chancellor, William H. Rendell, hosts a weekly lunchtime program called "Response to the Obama White House


In [7]:
print(dialog('python', 'What is python for?'))

torch.int64
<pad> Python is a Python programming language that can be used to build and write Python code. It provides an extensible and extensible framework for Python, and allows for the development and testing of Python code in an open-source environment. Python is often used to generate Python code that uses the Python programming language. It can be used to create dynamically generated Python code and provides a powerful graphical interface for writing and debugging Python code. Python also provides a lot of features for creating custom and dynamic Python code, such as: - Automatically generated Python code: Python provides the ability to create and implement new Python code, including features like: - Custom code: Pythonia supports the development of custom Python code, such that it can be written with a programming language like Python or Pascal, and includes the ability to build custom Python code in Python or Pascal. - Flexible code: Pythania provides a variety of


In [8]:
print(dialog('formel1', 'Who drives in F1?'))

torch.int64
<pad> I'm an English English based English speaking professional who is currently working as a technical consultant for a British automotive company. I'm a current member of F1's steering committee, and I have been in the industry for several years. I'm currently employed by the F1 team as a consultant. I'm looking for experienced drivers to help me with my tasks. I'm also an experienced engineer and have been in the automotive industry for several years, primarily for automotive solutions. I'm interested in working with clients, providing information and advice, and providing guidance and assistance. I'm available to answer any questions you may have, and I can help you with any further tasks. I'll be available at any time to discuss any issues you may have. I'm always available for help or assistance. I'd be happy to offer help or assistance, and I can assist you in any way I can. I'm here


In [9]:
print(dialog('formel1', 'how many races are in one season of formula 1?'))

torch.int64
<pad> For every race in the formula 1, for every season of formula 1, for every race in formula 2, for every season of Formula 1, for every season in Formula 2, for every season in formula 4, for every season in a cycle, for every season in every formula 6, for every season in season 7, for every season in each cycle, for every year in the cycle, for every cycle in each cycle, in formula 7, for every season of every cycle, for every consecutive year in a cycle. For example, if you're in Formula 2, for example, there are seven seasons in formula 1, for formula 2: Formula 3: Formula 4: Formula 5: Formula 6: Formula 8: Formula 9: Formula 9: formula 8: Formula 9 8: Formula 9 - Formula 9 - F1 - F1 12: Formula 13 - Formula 16 - Formula 14 - Formula 15 - Formula 17 - Formula 16</s>


In [10]:
print(dialog('word', 'How can I use microsoft word?'))

torch.int64
<pad> You can use any program and any operating system. First, you can use the “File” option to open and run a file. This will allow you to start writing the file, or run a command to run the file. To use a command, you can use “File” or “File”. Here are some steps to get started with Microsoft Word: 1. Start a new project: Start a new document in a new document and install it on your computer. 1. Open the document with the “File”. 2. Open the document in a folder and click the “File”, which will open the document in e.g. “File,” “File”, “File,” or “File”). 3. Open the document in an Excel document: Click the “File,” which will open the Excel document in a Excel document in e,g. “FILE,” or “FILE”). 4. Open the Excel


In [11]:
print(dialog('word', 'Do I have to buy it to use it?'))

torch.int64
<pad> Yes, you can download and install the program. To begin, you can download the software from the Google Play Store. Download the application and install the software on your computer. This will open the program and launch it. It is important to note that you should consider purchasing the program and installed it on your computer before using it, since it requires additional permissions. It's important to note that it's important to make sure that the software you use is compatible with your operating system. Once you install the software, you'll be able to run it at any time using it. You may be able to use it from a web browser or from an app store, such as Microsoft Windows. You may also be able to purchase the software by signing in with your Google account. If you want to use it on a mobile device, you'll need to pay a subscription to the service. If you want, you can choose to use the program on the


In [12]:
print(dialog('word', 'Where can I buy it?'))

torch.int64
<pad> if you want to download the program from the Google Play Store. To start, you can download the program from a website. You can purchase it from the website by using the "Get My Office" link. It is not necessary to purchase the software from the Google Play store. To begin, you can download and install the program from the Go-Store. This will download the application and install it on your computer. To begin, the application is installed on your computer. You can choose to download the program on the Google Play Store by clicking the "Get My App" link. You can choose from the following: - - - The Microsoft Office, available on Google Play Store, is available on the Microsoft Office website: https://www.microsoftoffice.com/ - The Microsoft Outlook, available on Google Exchange: https://www2.microsoftoffice.com - The Microsoft Word, available on Google Marketplace: https://www3.microsoftoffice.com There are
